In [1]:
% matplotlib inline


In [2]:
import torch
import math
from torch import nn
from torch.nn import functional as F
from torch.distributions import Normal, Laplace
import matplotlib.pyplot as plt
from torch import optim
from tqdm import tqdm

import seaborn as sns

# Exercise Sheet 2 - Practical: Normalizing Flows using PyTorch

In [ ]:
# def p_source(z, sigma=1., mu=0.):
#     pre_exp = 1.0 / torch.sqrt(2 * torch.pi * (sigma ** 2))
#     inner_squared = torch.pow(-0.5 * ((z - mu) / sigma), 2.0)
#     return pre_exp * torch.exp(inner_squared)


For this exercise we are given two distributions. 

First, a univariate Gaussian distribution, as a simple source distribution with density:
$$ p_{\mathrm{source}}(z)=\frac{1}{\sqrt{2\pi \sigma^2} } \exp \left(-\frac{1}{2}\left(\frac{z - \mu}{\sigma}\right)^2 \right)  \ \ \text{ with } \mu=0, \sigma=1  $$

Second, a univariate Laplace distribution, as the target distribution with density:
$$ p_{\mathrm{target}}(x)=\frac{1}{2b} \exp\left(-\frac{|x-\mu|}{b}\right) \ \ \text{ with } \mu=5, \sigma=3 $$

The essential goal of any flow model is to learn a function $f$ that can transform between two distributions.
That means we want to find an $f$ such that we can represent $p_\mathrm{source}$ using only $p_\mathrm{target}$ and $f$ and vice-versa represent $p_\mathrm{target}$ using only $p_\mathrm{source}$ and $f^{-1}$.
In practice, $p_\mathrm{target}$ is often a very complex distribution that we may not know the density of.
This could be the distribution of celebrity faces for example. 
What we do have however is a bunch of samples from that complex distribution that we can use to learn $f$. 

In our case, the target distribution is a univariate Laplace distribution and we have samples from that distribution. Your task in this exercise is to program crucial components of learning the parameters of the function $f$ that transforms samples from the target distribution (univariate Laplace) into samples from the source distribution (univariate Gaussian), and then use $f^{-1}$ to generate samples from the target distribution using samples from the source distribution.

## (1) A simple normalizing flow with a single affine layer

### [3 points]: (1.1) The Model

We will first implement a simple normalizing flow model that consists of only a single affine layer as $f_\theta$ (with $\theta=\{\alpha,\beta\}$):
$$ z = f_\theta(x) = \alpha  \cdot x + \beta $$ 

To implement our model, we need the following formulas:


$$
\begin{align*}
x = f_\theta^{-1}(z) &= \frac{z - \beta}{\alpha} \\ 
\log \left| \frac{df_\theta(x)}{dx} \right| &= \log |\alpha | \\ 
\log \left| \frac{df^{-1}_\theta(z)}{dz} \right| &= \log |\alpha |
\end{align*}
$$

Implement the simple affine normalizing flow model by completing the forward function below:

In [5]:
class Affine(nn.Module):
    def __init__(self):
        super().__init__()
        self.alpha = nn.Parameter(torch.FloatTensor(1).normal_())
        self.beta = nn.Parameter(torch.FloatTensor(1).normal_())

    def forward(self, x, log_d=0, inverse=False):
        if not inverse:
            z = self.alpha * x + self.beta
            log_d = log_d + torch.log(torch.abs(self.alpha))
        else:
            z = (x - self.beta) / self.alpha
            log_d = log_d + torch.log(torch.abs(((-1.0) * self.beta) / self.alpha))
        return z, log_d

    def __repr__(self):
        return "Affine(alpha={alpha:.2f}, beta={beta:.2f})".format(alpha=self.alpha[0], beta=self.beta[0])

Hint: `forward` must compute $f_\theta(x)$ and $f_\theta^{-1}(z)$. 
At the same time `forward` also returns the value $\log \left|\frac{df_\theta(x)}{dx}\right|$ added to `log_d` (this sum will become important later when we compose multiple functions). 

### [3 points]: (1.2) The objective function

Now that we have our basic flow model, let's implement the objective function that we want to optimize to learn the parameters of our flow model. The objective is maximizing the log-likelihood (or equivalently minimizing the negative log-likelihood) of our target distribution on samples that come from this distribution.

$$ \mathcal{L} = -\log p_\mathrm{target}(x)\\ p_\mathrm{target}(x) = p_\mathrm{source}(f(x)) \cdot \left| \frac{d f(x)}{dx} \right| \\
\Rightarrow - \log p_\mathrm{target}(x) = - \log p_\mathrm{source}(f(x)) - \log \left| \frac{d f(x)}{dx} \right|$$


Implement the objective function by completing the following code:

In [6]:
def p_source(z, sigma=1., mu=0.):
    pre_exp = 1.0 / torch.sqrt(2 * torch.pi * (sigma ** 2))
    inner_squared = torch.pow(-0.5 * ((z - mu) / sigma), 2.0)
    return pre_exp * torch.exp(inner_squared)

def objective(x, flow, avg=True): # TODO: avg unused
    z, log_d = flow(x)  # flow is an instance of a flow model, e.g. Affine()
    log_p_source = torch.log(p_source(z))  # log p_source(z)
    loss = (-1.) * log_p_source - log_d # TODO: Multivariate summierung?
    return loss

### [1 point] (1.2) Optimization
Let us use gradient descent to optimize our objective function. `pytorch` takes care of most things for us, e.g. it computes the gradients for us. Fill in the missing code to optimize the objective function using the right `pytorch` methods. The function takes in the training data $x$ from $p_\mathrm{target}$, the flow model, the objective and number of iterations.

In [ ]:
def optimize(x, flow, objective, iterations=2000):
    opt = optim.Adam(flow.parameters(), lr=1e-2)
    scheduler = optim.lr_scheduler.StepLR(opt, step_size=500, gamma=0.8)
    neg_log_likelihoods = []
    for _ in tqdm(range(iterations)):
        opt.zero_grad(False)  # zero out gradients first on the optimizer
        neg_log_likelihood = objective(x, flow)  # use the `objective` function
        neg_log_likelihood.backward()  # backpropagate the loss
        opt.step()
        scheduler.step()
        neg_log_likelihoods.append(neg_log_likelihood.detach().numpy())
    return neg_log_likelihoods

### [1 point]: (1.4) Training 
Now we can bring all the components together to train our flow model. However, we do not have any training data yet. 
The good thing about the simple example we are considering here is that we know the PDF for our target distribution and can therefore use this to generate samples that we can use as our training data.
Thus, in the following, we ask you to generate the training data, and use this as well as the components we defined above to train our normalizing flow. 

*Hint:* Torch distirbutions have a useful function called `sample()` that allows you to draw a number of sample from them.

In [ ]:
flow = ...  # Initialize the affine flow model


def train(flow, train_size=2000):
    p_target = Laplace(5, 3)
    x = ...  # Generate training data

    neg_log_likelihoods = ...  # Run training

    # Plot neg_log_likelihoods over training iterations:
    with sns.axes_style('ticks'):
        plt.plot(neg_log_likelihoods)
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
    sns.despine(trim=True)


train(flow)

### [1 point]: (1.5) Evaluation
Now that we have trained our first flow model, we want to inspect how well it works. For this purpose, we will now generate new samples from our target distribution $p_\mathrm{target}$ and compare them to the true samples we used for training.

In [ ]:
def evaluate(flow):
    p_source = Normal(0, 1)
    p_target = Laplace(5, 3)
    x_true = p_target.sample((2000, 1))  # samples to compare to

    # Generate samples from source distribution
    z = ...

    # Use our trained model get samples from the target distribution
    x_flow = ...

    # Plot histogram of training samples `x` and generated samples `x_flow` to compare the two.
    with sns.axes_style('ticks'):
        fig, ax = plt.subplots(figsize=(4, 4), dpi=150)
        ax.hist(x_true.detach().numpy().ravel(), bins=50, alpha=0.5, histtype='step');
        ax.hist(x_flow.detach().numpy().ravel(), bins=50, alpha=0.5, histtype='step');
        plt.xlabel("x")
        plt.ylabel("Num Samples")
        plt.legend()
    sns.despine(trim=True)

## (2) Flow with composition of mutiple layers with nonlinearities: neural networks!

### [3 points]: (2.1) A non-linear layer

After trying out our simple flow with a single affine layer above, we are now going to make our model a little more powerful by composing multiple simple functions.
For this, we first introduce a new simple function of the form:

 $$g(x)=\left\{\begin{array}{cc}{x} & {x>0} \\ {e^{x}-1} & {x \leqslant 0}\end{array}\right.$$
 

Now, similar to the affine layer we defined above, we now need to derive three components:

For the inverse:
$$
  g^{-1}(z) = \left\{\begin{array}{cl}{z} & {z>0} \\ {\log (z+1)} & {z \leqslant 0}\end{array}\right.
$$

For the derivative:
$$
\frac{d g}{d x}=\left\{\begin{array}{ll}{1} & {x>0} \\ {e^{x}} & {x \leqslant 0}\end{array}\right.
$$

For the derivative of the inverse:
$$
\frac{d g^{-1}}{d z}=\left\{\begin{array}{ll}{1} & {z>0} \\ {\frac{1}{z+1}} & {z \leqslant 0}\end{array}\right.
$$

Implementing this also follows the same pattern as for the affine layer. Fill in the forward function here:

In [ ]:
class NonLinear(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, logdet, inverse=False):
        if not inverse:
            z = ...
            ld = ...
        else:
            z = ...
            ld = ...
        logdet = logdet + ld
        return z, logdet

### [2 points]: (2.2) Stacking multiple layers

Now that we introduced a second type of layer, we can build more complex flow networks by stacking the individual layers on top of each other.
This means we want to use the following function:
$$
    g \circ f_{\theta_k} \circ ... \circ g \circ f_{\theta_1} = g(f_{\theta_k}(...g(f_{\theta_1}(x))...))
$$
Looking closely, we can see that this is simply a chained application of change of variables. 
Thus from the lecture, we can now the generalized objective function for a stacked network. 

$$
\mathcal{L} = \log p_\mathrm{source}(z) + \sum_{i=1}^{k} \log \left| \frac{df_i}{dz_{i-1}} \right|
$$

Now you should realise that the objective function we implemented earlier and the layers we implemented so far can already support the general case of composing multiple layers to a larger flow network. 
It does so by summing up the log-determinant term as we go through the layers in the forward pass.
What remains to do is the execution of the individual layers in the right order. 
Please implement this in the `forward` method of the following module that encapsulates all layers of our flow.

In [ ]:
class Flow(nn.Module):
    def __init__(self, layers=5):
        super().__init__()
        non_linear = NonLinear()
        self.layers = nn.ModuleList([])
        for _ in range(layers):
            self.layers.append(Affine())
            self.layers.append(non_linear)

    def forward(self, x, logdet=0, inverse=False):
        ...
        return x, logdet



Now we have a more complex flow, let's test it out and evaluate our results:

In [ ]:
flow = Flow(layers=5)
train(flow)
evaluate(flow)